In [1]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime,date

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## **HR data 전처리**

In [2]:
df = pd.read_csv('./HRDataset_v14.csv', delimiter=',')
df.head(5)

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,...,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,...,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,...,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,...,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,...,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,...,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2


In [3]:
df.columns

Index(['Employee_Name', 'EmpID', 'MarriedID', 'MaritalStatusID', 'GenderID',
       'EmpStatusID', 'DeptID', 'PerfScoreID', 'FromDiversityJobFairID',
       'Salary', 'Termd', 'PositionID', 'Position', 'State', 'Zip', 'DOB',
       'Sex', 'MaritalDesc', 'CitizenDesc', 'HispanicLatino', 'RaceDesc',
       'DateofHire', 'DateofTermination', 'TermReason', 'EmploymentStatus',
       'Department', 'ManagerName', 'ManagerID', 'RecruitmentSource',
       'PerformanceScore', 'EngagementSurvey', 'EmpSatisfaction',
       'SpecialProjectsCount', 'LastPerformanceReview_Date', 'DaysLateLast30',
       'Absences'],
      dtype='object')

In [4]:
# 결측치 확인
df.isnull().sum()

Employee_Name                   0
EmpID                           0
MarriedID                       0
MaritalStatusID                 0
GenderID                        0
EmpStatusID                     0
DeptID                          0
PerfScoreID                     0
FromDiversityJobFairID          0
Salary                          0
Termd                           0
PositionID                      0
Position                        0
State                           0
Zip                             0
DOB                             0
Sex                             0
MaritalDesc                     0
CitizenDesc                     0
HispanicLatino                  0
RaceDesc                        0
DateofHire                      0
DateofTermination             207
TermReason                      0
EmploymentStatus                0
Department                      0
ManagerName                     0
ManagerID                       8
RecruitmentSource               0
PerformanceSco

In [5]:
# DateofTermination == None 인 경우: 퇴사자가 아님: EmploymentStatus == Active, Termd == 0 이다.
# df[df.DateofTermination.isnull()]
df[df.DateofTermination.isnull()].loc[:,['Termd','EmploymentStatus','DateofTermination']]

,Termd,EmploymentStatus,DateofTermination
0,0,Active,NaN
3,0,Active,NaN
5,0,Active,NaN
6,0,Active,NaN
7,0,Active,NaN
...,...,...,...
305,0,Active,NaN
306,0,Active,NaN
308,0,Active,NaN
309,0,Active,NaN


In [6]:
# 퇴사자가 아님: EmploymentStatus == Active, DateofTermination = 0 으로 변경
# 매니저 이름이  Webster Butler인 ManagerID = 0 으로 변경
df.fillna(0,inplace=True)
df[df.DateofTermination==0].loc[:,['Termd','EmploymentStatus','DateofTermination']]

,Termd,EmploymentStatus,DateofTermination
0,0,Active,0
3,0,Active,0
5,0,Active,0
6,0,Active,0
7,0,Active,0
...,...,...,...
305,0,Active,0
306,0,Active,0
308,0,Active,0
309,0,Active,0


In [7]:
df[df['ManagerID']==0][['ManagerName','ManagerID']]

,ManagerName,ManagerID
19,Webster Butler,0.0
30,Webster Butler,0.0
44,Webster Butler,0.0
88,Webster Butler,0.0
135,Webster Butler,0.0
177,Webster Butler,0.0
232,Webster Butler,0.0
251,Webster Butler,0.0


In [8]:
def age_cal(day):
    today = datetime.today().date()
    day = day[:6]+'19'+day[-2:]
    birth = datetime.strptime(day,"%m/%d/%Y").date()
    year = today.year - birth.year
    if today.month < birth.month:
        year-=1
    elif today.month == birth.month and today.day < birth.day:
        year-=1
    return year

def workday_cal(df):
    ed = df['DateofTermination']
    st = df['DateofHire']
    if ed==0:
        return 0
    else:
        return (datetime.strptime(ed,"%m/%d/%Y").date() - datetime.strptime(st,"%m/%d/%Y").date()).days

def last_not_assessed_day_cal(df):
    ed = df['DateofTermination']
    st = df['LastPerformanceReview_Date']
    if ed==0:
        return -1
    else:
        return (datetime.strptime(ed,"%m/%d/%Y").date() - datetime.strptime(st,"%m/%d/%Y").date()).days

# 날짜 관련 columns 형식 통일
def date_of_hire(df):
    return datetime.strptime(df['DateofHire'],"%m/%d/%Y").date()
    
def date_of_termination(df):
    if df['DateofTermination'] == 0:
        return '0'
    else:
        return datetime.strptime(df['DateofTermination'],"%m/%d/%Y").date()

def last_performance_review_date(df):
    return datetime.strptime(df['LastPerformanceReview_Date'],"%m/%d/%Y").date()

In [9]:
# 생년월일을 통한 만나이 column 생성
# 나이
df['Age'] = df['DOB'].apply(age_cal)

# 입사-퇴사 날짜을 사용한 퇴사자 근무 일수(Working Days) | 퇴사하지 않은 사람의 값은 0
# 근무일 수 vs 연차 -> 연차: 근무일 수 // 365
df['WD'] = df.apply(workday_cal,axis=1)

# 퇴사자가 마지막으로 평가 받은 후 퇴사하기까지의 기간 | 근무자의 값은 -1
df['LastAssessedDate'] = df.apply(last_not_assessed_day_cal,axis=1)

df[['Age','WD','LastAssessedDate']]

,Age,WD,LastAssessedDate
0,40,0,-1
1,48,444,113
2,35,447,132
3,35,0,-1
4,34,1884,218
...,...,...,...
306,38,0,-1
307,41,2583,27
308,44,0,-1
309,44,0,-1


In [10]:
#입사 날짜와 퇴사 날짜 type: datetime으로 변경 후 string으로 설정
df['DateofHire'] = df.apply(date_of_hire,axis=1)
df["DateofHire"] = df["DateofHire"].astype(str)

df['DateofTermination'] = df.apply(date_of_termination,axis=1)
df["DateofTermination"] = df["DateofTermination"].astype(str)

df['LastPerformanceReview_Date'] = df.apply(last_performance_review_date,axis=1)
df["LastPerformanceReview_Date"] = df["LastPerformanceReview_Date"].astype(str)

In [12]:
df[['DateofHire','DateofTermination','LastPerformanceReview_Date']]

,DateofHire,DateofTermination,LastPerformanceReview_Date
0,2011-07-05,0,2019-01-17
1,2015-03-30,2016-06-16,2016-02-24
2,2011-07-05,2012-09-24,2012-05-15
3,2008-01-07,0,2019-01-03
4,2011-07-11,2016-09-06,2016-02-01
...,...,...,...
306,2014-07-07,0,2019-02-28
307,2008-09-02,2015-09-29,2015-09-02
308,2010-04-10,0,2019-02-21
309,2015-03-30,0,2019-02-01


In [13]:
# 데이터 저장
df.to_csv("./New_HR_dataset_24.01.22.csv",index=False)